# News Aggregator

## Problem Statement

Will fill it now, focusing on the code as of now

## Code

### Importing required libraries

In [68]:
### Installing required packages/ libraries
!pip3 install langchain newsapi-python transformers datasets rouge-score sentence-transformers langchain_community

In [69]:
import os
import torch
from datetime import datetime, timedelta
from newsapi import NewsApiClient
import pandas as pd
from langchain.chains import TransformChain, SequentialChain
from langchain.llms import HuggingFacePipeline
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    pipeline,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

### Configuration

In [70]:
# Configuration
# NEWSAPI_KEY = "2e44f6aa50bc4b6ab70dca723a81186d"
NEWSAPI_KEY = "d4db0b297bc5417ebe3612a8288d1863"
# TOPICS = ["security systems", "camera access", "voip", "broadband", "phone services"]
TOPICS = ["technology", "business", "health", "politics", "sports", "science", "entertainment", "climate change", "artificial intelligence", "cryptocurrency"]
# TOPICS = [
#     # Security Systems
#     "home security", "security solutions", "smart security", "alarm systems",
#     "CCTV", "intrusion detection", "access control", "security technology",
#     "surveillance", "security equipment", "monitoring systems",

#     # Camera Access
#     "remote monitoring", "camera surveillance", "IP cameras",
#     "video surveillance", "smart cameras", "surveillance cameras",
#     "home monitoring", "live camera feed", "camera security solutions",

#     # VoIP (Voice Over IP)
#     "internet telephony", "IP telephony", "VoIP solutions",
#     "softphones", "VoIP communication", "VoIP technology",
#     "internet calling", "voice communication", "digital telephony",
#     "cloud telephony",

#     # Broadband
#     "high-speed internet", "fiber internet", "broadband services",
#     "internet service providers", "Wi-Fi", "broadband connectivity",
#     "network solutions", "internet technology", "ISP services",

#     # Phone Services
#     "telecommunication services", "mobile phone services",
#     "landline services", "business telephony", "phone systems",
#     "communication services", "voice services", "telecom solutions",

#     # General Keywords
#     "Lisbon Creek technology", "small business security", "commercial security",
#     "smart home devices", "IoT in security", "telecom trends",
#     "cloud security", "network monitoring"
# ]
RELEVANCE_THRESHOLD = 0.01

# Initialize NewsAPI
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

### Fetching Articles from NewsAPI

In [71]:
# Fetching articles from last 3 days from NewsAPI for the topics related to Client
def fetch_tech_articles():
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

    all_articles = []
    for topic in TOPICS:
        response = newsapi.get_everything(
            q=topic,
            from_param=start_date,
            to=end_date,
            language='en',
            sort_by='relevancy',
            page_size=100  # Maximum as it is in developer mode
        )
        all_articles.extend(response['articles'])

    return pd.DataFrame(all_articles)

### Relevance checking for articles fetched from NewsAPI

In [72]:
# Zero-shot relevance check to check the relevance of the articles fetched from NewsAPI
class RelevanceChecker:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.topic_descriptions = [
            "Technical articles about physical or digital security systems",
            "News about camera technology and surveillance systems",
            "Voice over IP services and related technologies",
            "Broadband internet infrastructure and services",
            "Mobile phone services and telecommunications"
        ]
        self.topic_embeddings = self.model.encode(self.topic_descriptions)

    def is_relevant(self, text, threshold=RELEVANCE_THRESHOLD):
        if not text or len(text) < 100:  # Skip short content
            return False
        text_embedding = self.model.encode([text])
        similarity = cosine_similarity(text_embedding, self.topic_embeddings)
        return similarity.max() > threshold

# Initialize once
relevance_checker = RelevanceChecker()

### Preprocessing

In [73]:
def preprocess_articles(df):
    # Clean and deduplicate
    df = df.drop_duplicates(subset=['title'])
    df = df[df['content'].notna()]
    df['content'] = df['content'].str.split('…').str[0].str.strip()
    # return df['content']
    # Filter relevant articles
    df['is_relevant'] = df['content'].apply(relevance_checker.is_relevant)
    return df[df['is_relevant']].reset_index(drop=True)

### Loading the T5 models (T5-small, T5-base, distilT5)

In [74]:
def load_summarization_models():
    # Load T5-Small
    t5_small = pipeline(
        "text2text-generation",
        model="t5-small",
        device=0 if torch.cuda.is_available() else -1
    )

    #Load T5-Base
    t5_base = pipeline(
        "text2text-generation",
        model="t5-base",
        device=0 if torch.cuda.is_available() else -1
    )

    # Load DistilT5
    # distil_t5 = pipeline(
    #     "text2text-generation",
    #     model="mrm8488/distilT5-base-finetuned-summarize-news",
    #     device=0 if torch.cuda.is_available() else -1
    # )
    # distil_t5 = pipeline(
    #     "text2text-generation",
    #     model="dropout05/lfom_distilt5_realnewslike",
    #     device=0 if torch.cuda.is_available() else -1)

    return {
        "t5_small": t5_small,
        "t5_base": t5_base,
        # "distil_t5": distil_t5
    }

### Summarizing the articles

In [75]:
def summarize_articles(df, models):
    results = []
    for idx, row in df.iterrows():
        try:
            summaries = {
                "title": row["title"],
                "url": row["url"],
                "published": row["publishedAt"]
            }
            for model_name, model in models.items():
                summary = model(
                    "summarize: " + row["content"],
                    max_length=150,
                    min_length=30,
                    truncation=True
                )[0]['generated_text']
                summaries[model_name] = summary
            results.append(summaries)
        except Exception as e:
            print(f"Error processing {row['title']}: {str(e)}")
    return pd.DataFrame(results)

### Finetuning the model (Optional, not doing it for now)

In [76]:
def fine_tune_model(model_name, dataset):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    def preprocess(examples):
        inputs = tokenizer("summarize: " + examples["article"], max_length=512, truncation=True, padding="max_length")
        labels = tokenizer(examples["summary"], max_length=150, truncation=True, padding="max_length")
        return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": labels["input_ids"]}

    tokenized_data = dataset.map(preprocess, batched=True)

    training_args = TrainingArguments(
        output_dir=f"./{model_name}_fine_tuned",
        num_train_epochs=3,
        per_device_train_batch_size=4,
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data,
    )
    trainer.train()
    model.save_pretrained(f"./{model_name}_fine_tuned")


### Final Execution

In [77]:
if __name__ == "__main__":
    # Step 1: Fetch and preprocess articles
    raw_df = fetch_tech_articles()
    preprocessed_df = preprocess_articles(raw_df)
    print(f"Fetched {len(preprocessed_df)} relevant articles.")

    # Step 2: Load summarization models
    models = load_summarization_models()

    # Step 3: Summarize articles with all models
    summaries_df = summarize_articles(preprocessed_df, models)
    summaries_df.to_csv("tech_news_summaries.csv", index=False)
    print("Summaries saved to 'tech_news_summaries.csv'.")

    # Step 4: Fine-tune models (optional, requires GPU)
    # Uncomment to fine-tune
    # fine_tune_model("t5-small", dataset)
    # fine_tune_model("t5-base", dataset)
    # fine_tune_model("mrm8488/distilT5-base-finetuned-summarize-news", dataset)

Fetched 372 relevant articles.


Device set to use cuda:0
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Summaries saved to 'tech_news_summaries.csv'.
